Ejercicio 5.A
Crea una nueva red neuronal para el problema del breast cancer

Ahora razona con cuál de las 2 redes te quedarías y que threshold elegirías para cada uno de ellos.

Para elegir debes mostrar gráficas, una al lado de la otra para comparar lo siguiente:

Gráficos de perdida, sensibilidad y especificidad durante el entrenamiento
Matriz de confusión con las métricas: Sensibilidad, Especificidad, VPP, VPN, Prevalencia
Threshold vs. (Sensibilidad y Especificidad)
Threshold vs. Informedness (Muestra en el label el máximo)
¿Con qué red te quedarías?

In [123]:
import tensorflow as tf
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import time
import numpy as np
from sklearn.metrics import roc_auc_score
from statsmodels.stats.proportion import proportion_confint

In [124]:
RANDOM_SEED = 42
tf.keras.utils.set_random_seed(RANDOM_SEED)

In [125]:
def get_data():
    data = load_breast_cancer()
    X = data.data
    y = data.target
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state=RANDOM_SEED)
    return X_train, X_test, y_train, y_test

In [126]:
def compile_fit(net, x, y, first_layer, optimizer, learning_rate):
    tf.keras.utils.set_random_seed(5)
    layers = net
    model = Sequential()

    for idx, layer in enumerate(layers):
        if idx == 0:
            model.add(Dense(layer, activation='relu', input_dim=first_layer))
        elif idx == len(layers) - 1:
            model.add(Dense(layer, activation='sigmoid'))
        else:
            model.add(Dense(layer, activation='relu'))

    model.compile(loss='binary_crossentropy', optimizer=optimizer(learning_rate=learning_rate),
                  metrics=['Recall'])

    X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=RANDOM_SEED, stratify=y)

    mark1 = time.time()
    history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=20, verbose=False)
    mark2 = time.time()

    time_elapsed = mark2 - mark1

    return model, history, time_elapsed

In [127]:
def get_matrix(y_true, y_score, threshold=.5):
    y_pred = y_score >= threshold

    tp = np.sum((y_true == 1) & (y_pred == 1))
    tn = np.sum((y_true == 0) & (y_pred == 0))
    fp = np.sum((y_true == 0) & (y_pred == 1))
    fn = np.sum((y_true == 1) & (y_pred == 0))
    auc = roc_auc_score(y_true, y_score)

    return tp, tn, fp, fn, auc

In [128]:
def get_metrics(tp, tn, fp, fn):
    prevalence = (tp + fn) / (tp + tn + fp + fn)
    sensibility = tp / (tp + fn)
    specificity = tn / (fp + tn)
    vpp = tp / (tp + fp)
    vpn = tn / (tn + fn)
    accuracy = (tp + tn) / (tp + tn + fp + fn)
    informedness = sensibility + specificity - 1
    f1_score = 2 / ((1 / sensibility) + (1 / vpp))

    return prevalence, sensibility, specificity, vpp, vpn, accuracy, informedness, f1_score

In [129]:
def get_metrics_intervals(tp, tn, fp, fn):
    prevalence = proportion_confint((tp + fn), (tp + tn + fp + fn), alpha=0.05, method='wilson')
    sensibility = proportion_confint(tp, (tp + fn), alpha=0.05, method='wilson')
    specificity = proportion_confint(tn, (fp + tn), alpha=0.05, method='wilson')
    vpp = proportion_confint(tp, (tp + fp), alpha=0.05, method='wilson')
    vpn = proportion_confint(tn, (tn + fn), alpha=0.05, method='wilson')
    accuracy = proportion_confint((tp + tn), (tp + tn + fp + fn), alpha=0.05, method='wilson')
    auc = proportion_confint((tp + tn), (tp + tn + fp + fn), alpha=0.05, method='wilson')

    return prevalence, sensibility, specificity, vpp, vpn, accuracy, auc

In [130]:
X_train, X_test, y_train, y_test = get_data()

In [131]:
net_1 = [32, 64, 128, 64, 32, 1]
model_1, history_1 = compile_fit(net_1, X_train, y_train, X_train.shape[1], tf.keras.optimizers.Adam, 0.001)[0:2]

In [132]:
net_2 = [10, 20, 30, 40, 50, 40, 30, 20, 10, 1]
model_2, history_2 = compile_fit(net_2, X_train, y_train, X_train.shape[1], tf.keras.optimizers.Adam, 0.001)[0:2]

In [133]:
y_pred_1 = model_1.predict(X_test).reshape(-1)
tp_1, tn_1, fp_1, fn_1, auc_1 = get_matrix(y_test, y_pred_1)
prevalence_1, sensibility_1, specificity_1, vpp_1, vpn_1, accuracy_1, informedness_1, f1_score_1 = get_metrics(
    tp_1, tn_1, fp_1, fn_1)

prevalence_interval_1, sensibility_interval_1, specificity_interval_1, vpp_interval_1, vpn_interval_1, accuracy_interval_1, auc_interval_1 = get_metrics_intervals(
    tp_1, tn_1, fp_1, fn_1)

print("Model 1 metrics")
print(f'Prevalence: {prevalence_1} {prevalence_interval_1}')
print(f'Sensibility: {sensibility_1} {sensibility_interval_1}')
print(f'Specificity: {specificity_1} {specificity_interval_1}')
print(f'VPP: {vpp_1} {vpp_interval_1}')
print(f'VPN: {vpn_1} {vpn_interval_1}')
print(f'Accuracy: {accuracy_1} {accuracy_interval_1}')
print(f'Informedness: {informedness_1}')
print(f'AUC: {auc_1} {auc_interval_1}')
print(f'F1 Score: {f1_score_1}')

4/4 [==============================] - 0s 835us/step
Model 1 metrics
Prevalence: 0.6228070175438597 (0.5312021858550436, 0.7064051917199777)
Sensibility: 0.971830985915493 (0.9029849988335054, 0.9922407364173166)
Specificity: 0.9302325581395349 (0.8139101309875801, 0.9759884104227855)
VPP: 0.9583333333333334 (0.8845072325868389, 0.9857291925330748)
VPN: 0.9523809523809523 (0.8421014760304114, 0.9868424750910944)
Accuracy: 0.956140350877193 (0.9014191976891217, 0.981122490446672)
Informedness: 0.9020635440550278
AUC: 0.9947592531935802 (0.9014191976891217, 0.981122490446672)
F1 Score: 0.9650349650349651


In [137]:
y_pred_2 = model_2.predict(X_test).reshape(-1)
tp_2, tn_2, fp_2, fn_2, auc_2 = get_matrix(y_test, y_pred_2)
prevalence_2, sensibility_2, specificity_2, vpp_2, vpn_2, accuracy_2, informedness_2, f1_score_2 = get_metrics(
    tp_2, tn_2, fp_2, fn_2)

prevalence_interval_2, sensibility_interval_2, specificity_interval_2, vpp_interval_2, vpn_interval_2, accuracy_interval_2, auc_interval_2 = get_metrics_intervals(
    tp_2, tn_2, fp_2, fn_2)

print("Model 2 metrics")
print(f'Prevalence: {prevalence_2} {prevalence_interval_2}')
print(f'Sensibility: {sensibility_2} {sensibility_interval_2}')
print(f'Specificity: {specificity_2} {specificity_interval_2}')
print(f'VPP: {vpp_2} {vpp_interval_2}')
print(f'VPN: {vpn_2} {vpn_interval_2}')
print(f'Accuracy: {accuracy_2} {accuracy_interval_2}')
print(f'Informedness: {informedness_2}')
print(f'AUC: {auc_2} {auc_interval_2}')
print(f'F1 Score: {f1_score_2}')

4/4 [==============================] - 0s 1ms/step
Model 2 metrics
Prevalence: 0.6228070175438597 (0.5312021858550436, 0.7064051917199777)
Sensibility: 0.971830985915493 (0.9029849988335054, 0.9922407364173166)
Specificity: 0.9069767441860465 (0.783972963587464, 0.9632283593682872)
VPP: 0.9452054794520548 (0.8674100245784024, 0.9784875025843027)
VPN: 0.9512195121951219 (0.8386102112962913, 0.9865190546348309)
Accuracy: 0.9473684210526315 (0.8899118941026302, 0.9756578384920905)
Informedness: 0.8788077301015393
AUC: 0.9934490664919751 (0.8899118941026302, 0.9756578384920905)
F1 Score: 0.9583333333333334
